In [1]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.0f}'.format)

import f_validate_g4s as ed


from time import time
from datetime import timedelta

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

Client Scheduler: tcp://127.0.0.1:57248 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [3]:
path = "D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/"

In [4]:
test_dates = pd.date_range(start='01/01/2019',end='12/31/2019')

In [5]:
mapp = pd.read_pickle(path+'Actual/89 TID Br map.pkl')
#211 branches with just one ATM, can be treated as an ATM i/o Branch
mapp[mapp.TID!=mapp.TID_BR].BR_TIDCOUNT.value_counts()

2    256
1    211
3     42
4      8
Name: BR_TIDCOUNT, dtype: int64

In [6]:
thresh = pd.read_pickle(path+'89 Threshold Recommendations.pkl')
thresh = thresh[['tid','thresh']].drop_duplicates()
thresh.tid = thresh.tid.astype(str)
thresh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459 entries, 0 to 160758
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tid     459 non-null    object 
 1   thresh  459 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.8+ KB


In [7]:
thresh

,tid,thresh
0,91010033,600000
351,91010053,700000
702,91010068,200000
1053,91010075,500000
1404,91010090,900000
...,...,...
159354,BR398,500000
159705,BR400,1600000
160056,BR404,500000
160407,BR405,2800000


In [8]:
reload_df = pd.read_pickle(path+'Actual/80 Replenishment.pkl')
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['Pre-Emptive'] = reload_df['Pre-Emptive'].str.strip().fillna("NORMAL LOADING")
reload_df['date_str'] = reload_df.DATE.astype(str)


reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(str).str.strip().str.replace("","")
select = reload_df['Current Load Amount']==""
reload_df.loc[select,'Current Load Amount'] = "0"
reload_df['Current Load Amount'] = reload_df['Current Load Amount'].astype(int)

reload_df = reload_df[reload_df.DATE.isin(test_dates)].copy()
reload_df = reload_df.sort_values(['TID','DATE'],ignore_index=True)
reload_df['TID_BR'] = reload_df.TID.map(mapp.set_index('TID').TID_BR.to_dict())

first_reload = reload_df.drop_duplicates(subset=(['TID'])).copy()

reload_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194966 entries, 0 to 194965
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Last Load Amount     194965 non-null  object        
 1   Current Load Amount  194966 non-null  int32         
 2   % Cash Return/Atm    194966 non-null  float64       
 3   Replen               194966 non-null  object        
 4   Pre-Emptive          194966 non-null  object        
 5   TID                  194966 non-null  object        
 6   DATE                 194966 non-null  datetime64[ns]
 7   date_str             194966 non-null  object        
 8   TID_BR               120787 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(6)
memory usage: 12.6+ MB


In [9]:
preempt = reload_df[reload_df['Pre-Emptive'] =='PRE-EMPTIVE DUE TO CF'][['date_str','TID_BR','Pre-Emptive','Current Load Amount']]

In [10]:
df = pd.concat([pd.read_csv(path+'Prophet Forecast/'+i, dtype={'ds':'str','pred_date':'str','tid':'str'}) 
                for i in os.listdir(path+'Prophet Forecast/')],ignore_index=True)
df.sort_values(['tid','pred_date','ds'], inplace=True)
df['pred_date_D'] = pd.to_datetime(df.pred_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2255526 entries, 0 to 2255525
Data columns (total 9 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Unnamed: 0   int64         
 1   ds           object        
 2   y            float64       
 3   yhat         float64       
 4   pred_date    object        
 5   pred_day     object        
 6   se           float64       
 7   tid          object        
 8   pred_date_D  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 172.1+ MB


In [11]:
df.pred_day

0           day 0
1           day 1
2           day 2
3           day 3
4           day 4
            ...  
2255521     day 9
2255522    day 10
2255523    day 11
2255524    day 12
2255525    day 13
Name: pred_day, Length: 2255526, dtype: object

In [12]:
actual = df.pivot_table(index=['tid','pred_date','pred_date_D'],columns='pred_day',values='yhat',aggfunc=sum).reset_index()
actual[['date_str','y']] = df[df.pred_day=='day 0'][['ds','y']].reset_index(drop=True)
actual

pred_day,tid,pred_date,pred_date_D,day 0,day 1,day 10,day 11,day 12,day 13,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,date_str,y
0,91010033,2018-12-30,2018-12-30,155508,265152,1851613,481672,433560,80921,186592,169479,356815,347041,70164,167317,421752,539719,2018-12-31,0
1,91010033,2018-12-31,2018-12-31,207538,185760,477653,431761,79208,69317,167528,354766,344174,68077,164862,419045,535998,1852163,2019-01-01,1000
2,91010033,2019-01-01,2019-01-01,184668,166044,430919,78654,68588,183367,353578,341692,66152,163814,417214,534695,1851578,475669,2019-01-02,500
3,91010033,2019-01-02,2019-01-02,163562,353159,77698,69293,182862,206244,343207,66352,163227,416627,532935,1850632,476869,430396,2019-01-03,13100
4,91010033,2019-01-03,2019-01-03,351581,342394,68069,183690,205364,168946,68746,165127,417774,533590,1849611,476500,431211,79022,2019-01-04,129000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161104,BR406,2019-12-11,2019-12-11,898535,1620265,533635,1170023,830529,0,740622,582667,934462,1106524,938056,1001953,1209971,728159,2019-12-12,1327500
161105,BR406,2019-12-12,2019-12-12,1636405,740727,1179011,844402,79230,925810,579870,932774,1106646,944141,1013050,1221949,733714,541999,2019-12-13,1654700
161106,BR406,2019-12-13,2019-12-13,738805,573301,839623,6500,930424,1485818,926539,1103846,943195,1014934,1225905,737567,541904,1196360,2019-12-14,1002500
161107,BR406,2019-12-14,2019-12-14,575064,927466,17192,925867,1482471,803028,1104235,941508,1012155,1222740,735538,537988,1186421,836384,2019-12-15,695000


In [13]:
atms = [atm for atm in actual.tid.unique() 
        if atm in mapp[mapp.BR_TIDCOUNT > 1].TID_BR.unique()]
len(atms)

66

##### First reload is generally 0-2days apart for atms within branches. assume that first reload for all is on the earliest date 


In [14]:
first_reload[first_reload.TID_BR.isin(atms)].sort_values('TID_BR').head(100)

,Last Load Amount,Current Load Amount,% Cash Return/Atm,Replen,Pre-Emptive,TID,DATE,date_str,TID_BR
66894,6200000,6200000,53,LOADING,PRE-EMPTIVE DUE TO CF,91016834,2019-01-04,2019-01-04,BR003
11863,6200000,6200000,43,LOADING,PRE-EMPTIVE DUE TO CF,91010267,2019-01-04,2019-01-04,BR003
87824,6200000,6200000,57,LOADING,PRE-EMPTIVE DUE TO CF,91017477,2019-01-04,2019-01-04,BR015
48587,6200000,6200000,51,LOADING,PRE-EMPTIVE DUE TO CF,91016074,2019-01-05,2019-01-05,BR015
99694,5200000,5200000,52,LOADING,PRE-EMPTIVE DUE TO CF,91017846,2019-01-04,2019-01-04,BR019
14077,5200000,5200000,68,LOADING,NORMAL LOADING,91010306,2019-01-02,2019-01-02,BR019
18019,6200000,6200000,88,LOADING,NORMAL LOADING,91010357,2019-01-02,2019-01-02,BR025
16416,6200000,6200000,54,LOADING,PRE-EMPTIVE DUE TO CF,91010337,2019-01-01,2019-01-01,BR025
11744,6200000,6200000,61,LOADING,PRE-EMPTIVE DUE TO CF,91010266,2019-01-03,2019-01-03,BR029
2093,6200000,6200000,58,LOADING,PRE-EMPTIVE DUE TO CF,91010055,2019-01-03,2019-01-03,BR029


In [15]:
# actual

In [16]:
def par(atm,threshold,rel_amt):
    
    atm_no = first_reload.loc[first_reload['TID_BR'] == atm].TID.nunique()

    date_o = first_reload.loc[first_reload['TID_BR'] == atm].reset_index().DATE[0]
    actual_atm = actual[(actual.tid== atm) & (actual.pred_date_D >= date_o-timedelta(days=1))] \
                    [['tid','pred_date','pred_date_D','date_str','y'] + window_col].reset_index(drop=True).copy()

    g4s=preempt[preempt.TID_BR==atm].set_index('date_str')
    g4s = g4s.groupby([g4s.index,'Pre-Emptive']).size().reset_index().set_index('date_str')
    g4s.columns=['Pre-Emptive','No']
    g4s['Current Load Amount'] = rel_amt*g4s.No

    if g4s.shape[0]==0: g4s=None
    
    sched_= ed.sched_branch(actual_atm,rel_amt=rel_amt,r=r,reload_charge=reload_charge,
                      t=threshold,atm_no=atm_no, hor=hor,window=window,g4s=g4s)
    sched_['fix_rel']=rel_amt
    ss=sched_.groupby('cog_preemptive') \
                        [['reload_flag','cashout_flag','thresh_breach_flag','necessary_flag',
                                             'civ_cost','reload_cost']].sum().reset_index()
    ss['tid']=atm
    ss['fix_rel'] = rel_amt
    bb = (sched_['total_bal'] / rel_amt).round().value_counts().reset_index()
    bb.columns = ['ATMs to Cover Start Bal','No of ATMs']
    
    sched_.to_csv(path+'Prophet Test Branch Thresh R/_sched_'+atm+'_'+str(rel_amt)[:2]+'.csv')
    ss.join(bb,how='outer').to_csv(path+'Prophet Test Perf Branch Thresh R/_schedperf_'+ atm+'_'+str(rel_amt)[:2]+'.csv')
    
    return sched_,ss

In [17]:
hor=2
window = 12
r = 0.05/365
reload_charge=2000
threshold=0.25
window_col = ['day '+ str(i) for i in range(window+hor)]

In [18]:
# a,b = par('BR003',6200000)

In [19]:
thresh = pd.read_pickle(path+'89 Threshold Recommendations.pkl')
thresh = thresh[['tid','thresh']].drop_duplicates()
thresh.tid = thresh.tid.astype(str)
thresh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459 entries, 0 to 160758
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tid     459 non-null    object 
 1   thresh  459 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.8+ KB


In [20]:
atm='BR003'


In [21]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].thresh.values[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=3200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:09:16.381165


In [23]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].thresh.values[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=4200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:05:59.090267


In [24]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].thresh.values[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=5200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:04:02.362824


In [25]:
dbatms = db.from_sequence(atms)
dbthresh = db.from_sequence([thresh[thresh.tid==atm].thresh.values[0] for atm in atms])
dbmaster = dbatms.map(par,dbthresh,rel_amt=6200000)

start=time()
df = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:03:19.708945
